Deep Learning
=============

Assignment 1
------------

The objective of this assignment is to learn about simple data curation practices, and familiarize you with some of the data we'll be reusing later.

This notebook uses the [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) dataset to be used with python experiments. This dataset is designed to look like the classic [MNIST](http://yann.lecun.com/exdb/mnist/) dataset, while looking a little more like real data: it's a harder task, and the data is a lot less 'clean' than MNIST.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import matplotlib.pyplot as plt
import numpy as np
import os
import sys
import tarfile
from IPython.display import display, Image
from scipy import ndimage
from sklearn.linear_model import LogisticRegression
from six.moves.urllib.request import urlretrieve
from six.moves import cPickle as pickle

# Config the matplotlib backend as plotting inline in IPython
%matplotlib inline

First, we'll download the dataset to our local machine. The data consists of characters rendered in a variety of fonts on a 28x28 image. The labels are limited to 'A' through 'J' (10 classes). The training set has about 500k and the testset 19000 labelled examples. Given these sizes, it should be possible to train models quickly on any machine.

In [0]:
url = 'http://commondatastorage.googleapis.com/books1000/'
last_percent_reported = None

def download_progress_hook(count, blockSize, totalSize):
  """A hook to report the progress of a download. This is mostly intended for users with
  slow internet connections. Reports every 1% change in download progress.
  """
  global last_percent_reported
  percent = int(count * blockSize * 100 / totalSize)

  if last_percent_reported != percent:
    if percent % 5 == 0:
      sys.stdout.write("%s%%" % percent)
      sys.stdout.flush()
    else:
      sys.stdout.write(".")
      sys.stdout.flush()
      
    last_percent_reported = percent
        
def maybe_download(filename, expected_bytes, force=False):
  """Download a file if not present, and make sure it's the right size."""
  if force or not os.path.exists(filename):
    print('Attempting to download:', filename) 
    filename, _ = urlretrieve(url + filename, filename, reporthook=download_progress_hook)
    print('\nDownload Complete!')
  statinfo = os.stat(filename)
  if statinfo.st_size == expected_bytes:
    print('Found and verified', filename)
  else:
    raise Exception(
      'Failed to verify ' + filename + '. Can you get to it with a browser?')
  return filename

train_filename = maybe_download('notMNIST_large.tar.gz', 247336696)
test_filename = maybe_download('notMNIST_small.tar.gz', 8458043)

Extract the dataset from the compressed .tar.gz file.
This should give you a set of directories, labelled A through J.

In [0]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import matplotlib.pyplot as plt
import numpy as np
import os
import sys
import tarfile
from IPython.display import display, Image
from scipy import ndimage
from sklearn.linear_model import LogisticRegression
from six.moves.urllib.request import urlretrieve
from six.moves import cPickle as pickle

# Config the matplotlib backend as plotting inline in IPython
%matplotlib inline

---
Problem 1
---------

Let's take a peek at some of the data to make sure it looks sensible. Each exemplar should be an image of a character A through J rendered in a different font. Display a sample of the images that we just downloaded. Hint: you can use the package IPython.display.

---

In [0]:
Image(filename="./notMNIST_large/A/a29ydW5pc2hpLnR0Zg==.png")

Now let's load the data in a more manageable format. Since, depending on your computer setup you might not be able to fit it all in memory, we'll load each class into a separate dataset, store them on disk and curate them independently. Later we'll merge them into a single dataset of manageable size.

We'll convert the entire dataset into a 3D array (image index, x, y) of floating point values, normalized to have approximately zero mean and standard deviation ~0.5 to make training easier down the road. 

A few images might not be readable, we'll just skip them.

In [0]:
image_size = 28  # Pixel width and height.
pixel_depth = 255.0  # Number of levels per pixel.

In [0]:
def load_letter(folder, min_num_images):
  """Load the data for a single letter label."""
  image_files = os.listdir(folder)
  dataset = np.ndarray(shape=(len(image_files), image_size, image_size),
                         dtype=np.float32)
  print(folder)
  num_images = 0
  for image in image_files:
    image_file = os.path.join(folder, image)
    try:
      image_data = (ndimage.imread(image_file).astype(float) - 
                    pixel_depth / 2) / pixel_depth
      if image_data.shape != (image_size, image_size):
        raise Exception('Unexpected image shape: %s' % str(image_data.shape))
      dataset[num_images, :, :] = image_data
      num_images = num_images + 1
    except IOError as e:
      print('Could not read:', image_file, ':', e, '- it\'s ok, skipping.')
    
  dataset = dataset[0:num_images, :, :]
  if num_images < min_num_images:
    raise Exception('Many fewer images than expected: %d < %d' %
                    (num_images, min_num_images))
    
  print('Full dataset tensor:', dataset.shape)
  print('Mean:', np.mean(dataset))
  print('Standard deviation:', np.std(dataset))
  return dataset
        
def maybe_pickle(data_folders, min_num_images_per_class, force=False):
  dataset_names = []
  for folder in data_folders:
    set_filename = folder + '.pickle'
    dataset_names.append(set_filename)
    if os.path.exists(set_filename) and not force:
      # You may override by setting force=True.
      print('%s already present - Skipping pickling.' % set_filename)
    else:
      print('Pickling %s.' % set_filename)
      dataset = load_letter(folder, min_num_images_per_class)
      try:
        with open(set_filename, 'wb') as f:
          pickle.dump(dataset, f, pickle.HIGHEST_PROTOCOL)
      except Exception as e:
        print('Unable to save data to', set_filename, ':', e)
  
  return dataset_names

train_datasets = maybe_pickle(train_folders, 45000)
test_datasets = maybe_pickle(test_folders, 1800)

In [0]:
os.getcwd()

In [0]:
train_datasets = ["notMNIST_large/"+nm for nm in sorted(filter(lambda n: n[-6:] == "pickle",os.listdir("./notMNIST_large")))]
test_datasets = ["notMNIST_small/"+nm for nm in sorted(filter(lambda n: n[-6:] == "pickle",os.listdir("./notMNIST_small")))]

---
Problem 2
---------

---

In [0]:
pickle_file = train_datasets[7]
with open(pickle_file, 'rb') as f:
    letter_set = pickle.load(f)  # unpickle
    sample_idx = np.random.randint(len(letter_set))  # pick a random image index
    sample_image = letter_set[sample_idx, :, :]  # extract a 2D slice
    plt.figure()
    plt.imshow(sample_image)  # display it

---
Problem 3
---------
Another check: we expect the data to be balanced across classes. Verify that.

---

In [0]:
for nm in (filter(lambda n: n[-6:] == "pickle",os.listdir("./notMNIST_large"))):
    f = open("./notMNIST_large/"+nm, 'rb')
    letter_set = pickle.load(f)
    print(letter_set.shape[0])

Merge and prune the training data as needed. Depending on your computer setup, you might not be able to fit it all in memory, and you can tune `train_size` as needed. The labels will be stored into a separate array of integers 0 through 9.

Also create a validation dataset for hyperparameter tuning.

In [0]:
def make_arrays(nb_rows, img_size):
  if nb_rows:
    dataset = np.ndarray((nb_rows, img_size, img_size), dtype=np.float32)
    labels = np.ndarray(nb_rows, dtype=np.int32)
  else:
    dataset, labels = None, None
  return dataset, labels

def merge_datasets(pickle_files, train_size, valid_size=0):
  num_classes = len(pickle_files)
  valid_dataset, valid_labels = make_arrays(valid_size, image_size)
  train_dataset, train_labels = make_arrays(train_size, image_size)
  vsize_per_class = valid_size // num_classes
  tsize_per_class = train_size // num_classes
    
  start_v, start_t = 0, 0
  end_v, end_t = vsize_per_class, tsize_per_class
  end_l = vsize_per_class+tsize_per_class
  for label, pickle_file in enumerate(pickle_files):       
    try:
      with open(pickle_file, 'rb') as f:
        letter_set = pickle.load(f)
        # let's shuffle the letters to have random validation and training set
        np.random.shuffle(letter_set)
        if valid_dataset is not None:
          valid_letter = letter_set[:vsize_per_class, :, :]
          valid_dataset[start_v:end_v, :, :] = valid_letter
          valid_labels[start_v:end_v] = label
          start_v += vsize_per_class
          end_v += vsize_per_class
                    
        train_letter = letter_set[vsize_per_class:end_l, :, :]
        train_dataset[start_t:end_t, :, :] = train_letter
        train_labels[start_t:end_t] = label
        start_t += tsize_per_class
        end_t += tsize_per_class
    except Exception as e:
      print('Unable to process data from', pickle_file, ':', e)
      raise
    
  return valid_dataset, valid_labels, train_dataset, train_labels
            
            
train_size = 200000
valid_size = 10000
test_size = 10000

valid_dataset, valid_labels, train_dataset, train_labels = merge_datasets(
  train_datasets, train_size, valid_size)
_, _, test_dataset, test_labels = merge_datasets(test_datasets, test_size)

print('Training:', train_dataset.shape, train_labels.shape)
print('Validation:', valid_dataset.shape, valid_labels.shape)
print('Testing:', test_dataset.shape, test_labels.shape)

Next, we'll randomize the data. It's important to have the labels well shuffled for the training and test distributions to match.

In [0]:
def randomize(dataset, labels):
  permutation = np.random.permutation(labels.shape[0])
  shuffled_dataset = dataset[permutation,:,:]
  shuffled_labels = labels[permutation]
  return shuffled_dataset, shuffled_labels
train_dataset, train_labels = randomize(train_dataset, train_labels)
test_dataset, test_labels = randomize(test_dataset, test_labels)
valid_dataset, valid_labels = randomize(valid_dataset, valid_labels)

---
Problem 4
---------
Convince yourself that the data is still good after shuffling!

---

In [0]:
sample_idx = np.random.randint(len(train_dataset))  # pick a random image index
sample_image = train_dataset[sample_idx, :, :]  # extract a 2D slice
plt.figure()
plt.imshow(sample_image)  # display it
print(train_labels[sample_idx])

Finally, let's save the data for later reuse:

In [0]:
pickle_file = 'notMNIST.pickle'

try:
  f = open(pickle_file, 'wb')
  save = {
    'train_dataset': train_dataset,
    'train_labels': train_labels,
    'valid_dataset': valid_dataset,
    'valid_labels': valid_labels,
    'test_dataset': test_dataset,
    'test_labels': test_labels,
    }
  pickle.dump(save, f, pickle.HIGHEST_PROTOCOL)
  f.close()
except Exception as e:
  print('Unable to save data to', pickle_file, ':', e)
  raise

In [0]:
statinfo = os.stat(pickle_file)
print('Compressed pickle size:', statinfo.st_size)

In [0]:
pickle_file = 'notMNIST.pickle'
f = open(pickle_file, 'rb')
pkl = pickle.load(f)
test_labels = pkl["test_labels"]
valid_labels = pkl["valid_labels"]
valid_dataset = pkl["valid_dataset"]
train_labels = pkl["train_labels"]
test_dataset = pkl["test_dataset"]
train_dataset = pkl["train_dataset"]
f.close()

---
Problem 5
---------

By construction, this dataset might contain a lot of overlapping samples, including training data that's also contained in the validation and test set! Overlap between training and test can skew the results if you expect to use your model in an environment where there is never an overlap, but are actually ok if you expect to see training samples recur when you use it.
Measure how much overlap there is between training, validation and test samples.

Optional questions:
- What about near duplicates between datasets? (images that are almost identical)
- Create a sanitized validation and test set, and compare your accuracy on those in subsequent assignments.
---

In [0]:
(train_dataset[4] == train_dataset[5])

In [0]:
train_idx_noeql = list(filter(lambda i1:
			      not(any(filter(lambda i2: (train_dataset[i1]==
					     test_dataset[i2]).all(),
					 range(test_dataset.shape[0])))),
			      range(train_dataset.shape[0])))

In [0]:
train_idx_eql = list(filter(lambda i: not(i in train_idx_noeql), 
		       range(train_dataset.shape[0])))

In [0]:
train_test_idx_eql = [(i,j)
		      for i in train_idx_eql
		      for j in range(test_dataset.shape[0])
		      if (train_dataset[i]==test_dataset[j]).all()]

In [0]:
np.linalg.norm(test_dataset[1171]-test_dataset[1276])

#### working implementation start

In [0]:
pkl = np.load('notMNIST.pickle',mmap_mode='r')
test_labels = pkl["test_labels"]
valid_labels = pkl["valid_labels"]
valid_dataset = pkl["valid_dataset"]
train_labels = pkl["train_labels"]
test_dataset = pkl["test_dataset"]
train_dataset = pkl["train_dataset"]

In [0]:
try:    
    np.save('train_dataset',train_dataset)
except Exception as e:
    print('Unable to save data to', 'train_dataset', ':', e)
    raise
try:    
    np.save('test_dataset',test_dataset)
except Exception as e:
    print('Unable to save data to', 'test_dataset', ':', e)
    raise
try:    
    np.save('valid_dataset',valid_dataset)
except Exception as e:
    print('Unable to save data to', 'valid_dataset', ':', e)
    raise

In [2]:
train_dataset = np.load('train_dataset.npy', mmap_mode='r')
valid_dataset = np.load('valid_dataset.npy', mmap_mode='r')
test_dataset = np.load('test_dataset.npy', mmap_mode='r')

In [0]:
import multiprocessing
multiprocessing.cpu_count()
cpus = 4
pool = multiprocessing.Pool(processes=cpus)

In [0]:
import itertools
domain = itertools.product(range(train_dataset.shape[0]),
			   range(test_dataset.shape[0]))

In [0]:
import arrayfire as af

In [3]:
def is_apx_eql(ix, dataset_A, dataset_B):
    i,j = ix
    return np.linalg.norm(dataset_A[i]-dataset_B[j]) < 2e-2
def is_eps_eql(ix, dataset_A, dataset_B):
    i,j = ix
    return np.linalg.norm(dataset_A[i]-dataset_B[j]) < 1e-6
def is_exact_eql(ix, dataset_A, dataset_B):
    i,j = ix
    return np.linalg.norm(dataset_A[i]-dataset_B[j]) == 0

##### problem 5 sln 2

In [0]:
import multiprocessing
multiprocessing.cpu_count()
cpus = 1
pool = multiprocessing.Pool(processes=cpus)

In [4]:
import itertools
from functools import reduce

In [9]:
def diffs(dataset_A, dataset_B, domain, diff_eval):
  def func(ix_lst):
    return [(i,j) for i,j in ix_lst if diff_eval((i,j), dataset_A, dataset_B)];
  return list(map(func, domain))
# try to move to multiprocessing:  pool.imap(func, domain, chunksize=4)

In [6]:
domain_lsts = map(lambda i: itertools.product([i],
					      range(valid_dataset.shape[0])),
		  range(train_dataset.shape[0]))

In [10]:
st_tr_vl_apx_lsts = diffs(train_dataset, valid_dataset, domain_lsts, is_apx_eql)
tr_vl_apx_lst = list(reduce(lambda a,b: a+b, lst_tr_vl_apx_lsts))
lst_tr_vl_eps_lsts = diffs(train_dataset, valid_dataset, lst_tr_vl_apx_lsts,
			   is_eps_eql)
tr_vl_eps_lst = list(reduce(lambda a,b: a+b, lst_tr_vl_eps_lsts))
lst_tr_vl_eql_lsts = diffs(train_dataset, valid_dataset, lst_tr_vl_eps_lsts, is_exact_eql)
tr_vl_eql_lst = list(reduce(lambda a,b: a+b, lst_tr_vl_eql_lsts))

NameError: name 'lst_tr_vl_apx_lsts' is not defined

In [0]:
domain = itertools.product(range(test_dataset.shape[0]),
			   range(valid_dataset.shape[0]))
te_vl_apx_lst = diffs(test_dataset, valid_dataset, domain, is_apx_eql)
te_vl_eps_lst = diffs(test_dataset, valid_dataset, tr_vl_apx_lst, is_apx_eql)
te_vl_eql_lst = diffs(test_dataset, valid_dataset, tr_vl_eps_lst, is_apx_eql)

In [0]:
pickle_file = 'diffs_by_ix.pickle'
try:    
    np.save(pickle_file,diffs_by_ix)
    f.close()
except Exception as e:
    print('Unable to save data to', pickle_file, ':', e)
    raise

In [0]:
diffs_by_ix = np.load('diffs_by_ix.pickle.npy', mmap_mode='r')

##### one cpu

In [0]:
import itertools
domain = itertools.product(range(diffs_by_ix.shape[0]),range(diffs_by_ix.shape[1]))

In [0]:
apx_lst = [(i,j) for i,j in domain if diffs_by_ix[i,j] < 2e-2]

In [0]:
len(apx_lst)

In [0]:
eps_lst = [(i,j) for i,j in apx_lst if diffs_by_ix[i,j] < 1e-6]

In [0]:
pickle_file = 'eql_lsts.pickle'

try:
  f = open(pickle_file, 'wb')
  save = {
    'apx_lst': apx_lst,
    'eps_lst': eps_lst,
    'eql_lst': eql_lst,
    }
  pickle.dump(save, f, pickle.HIGHEST_PROTOCOL)
  f.close()
except Exception as e:
  print('Unable to save data to', pickle_file, ':', e)
  raise

In [0]:
eql_lst = [(i,j) for i,j in eps_lst if diffs_by_ix[i,j] == 0]

In [0]:
len(eql_lst)

In [0]:
sanitized

In [0]:
pickle_file = 'eql_lsts.pickle'
try:
  f = open(pickle_file, 'wb')
  save = {
    'apx_lst': apx_lst,
    'eps_lst': eps_lst,
    'eql_lst': eql_lst,
    }
  pickle.dump(save, f, pickle.HIGHEST_PROTOCOL)
  f.close()
except Exception as e:
  print('Unable to save data to', pickle_file, ':', e)
  raise

##### parallel

In [0]:
import multiprocessing
multiprocessing.cpu_count()
cpus = 4
pool = multiprocessing.Pool(processes=cpus)
import itertools
domain = itertools.product(range(diffs_by_ix.shape[0]),
			   range(diffs_by_ix.shape[1]))

In [0]:
def is_apx_eql(ix):
    i,j = ix
    return diffs_by_ix[i,j] < 2e-2, i, j
apx_eql = pool.imap(is_apx_eql,domain, chunksize=4)
gen = ((i,j) for ret, i, j in apx_eql if ret)
apx_lst = list(gen)

In [0]:
def is_eps_eql(ix):
    i,j = ix
    return diffs_by_ix[i,j] < 1e-6, i, j
eps_eql = pool.imap(is_eps_eql,apx_lst, chunksize=4)
gen = ((i,j) for ret, i, j in eps_eql if ret)
eps_lst = list(gen)

In [0]:
def is_eps_eql(ix):
    i,j = ix
    return diffs_by_ix[i,j] == 0, i, j
exact_eql = pool.imap(is_exact_eql, eps_lst, chunksize=4)
gen = ((i,j) for ret, i, j in exact_eql if ret)
eql_lst = list(gen)

In [0]:
pool.close()

#### other methods... not fully working

In [0]:
import arrayfire as af

In [0]:
af_train_dataset = af.Array(train_dataset.ctypes.data,
			    train_dataset.shape,
			    train_dataset.dtype.char)
af_test_dataset = af.Array(test_dataset.ctypes.data,
			   test_dataset.shape,
			   test_dataset.dtype.char)

In [0]:
diffs_by_ix = np.zeros((af_train_dataset.shape[0], af_test_dataset.shape[0]))
for i in range(af_train_dataset.shape[0]):
    for j in range(af_test_dataset.shape[0]):
        diffs_by_ix[i,j] = af.norm(af_train_dataset[i]-af_test_dataset[j])

In [0]:
import julia

In [0]:
jl = julia.Julia()

In [0]:
pickle_file = 'notMNIST_vars.pickle'
try:
  f = open(pickle_file, 'wb')
  save = {
    'i': i,
    'j': j,
    'diffs_by_ix': diffs_by_ix,
    }
  pickle.dump(save, f, pickle.HIGHEST_PROTOCOL)
  f.close()
except Exception as e:
  print('Unable to save data to', pickle_file, ':', e)
  raise

In [0]:
diffs_by_ix.shape

In [0]:
i,j = train_test_idx_eql[np.random.randint(len(train_test_idx_eql))]
print("i:"+str(i)+", j:"+str(j))
print(diffs_by_ix[i,j])

In [0]:
diffs_by_ix[4,299]

---
Problem 6
---------

Let's get an idea of what an off-the-shelf classifier can give you on this data. It's always good to check that there is something to learn, and that it's a problem that is not so trivial that a canned solution solves it.

Train a simple model on this data using 50, 100, 1000 and 5000 training samples. Hint: you can use the LogisticRegression model from sklearn.linear_model.

Optional question: train an off-the-shelf model on all the data!

---

In [2]:
import pickle
pickle_file = 'eql_lsts.pickle'
eql_lsts = np.load(pickle_file)
apx_eql_lst = eql_lsts["apx_lst"]

In [3]:
pkl = np.load('notMNIST.pickle',mmap_mode='r')
test_labels = pkl["test_labels"]
valid_labels = pkl["valid_labels"]
valid_dataset = pkl["valid_dataset"]
train_labels = pkl["train_labels"]
test_dataset = pkl["test_dataset"]
train_dataset = pkl["train_dataset"]

In [5]:
import itertools
import random
import sklearn.linear_model
bad_train_ix = map(lambda x: x[0], apx_eql_lst)
good_train_ix = list(filter(lambda i: i not in bad_train_ix, 
                            range(train_dataset.shape[0])))

In [6]:
def reservoir_sampling(iterable, r=1):
    "Random selection from itertools.permutations(iterable, r)"
    it = iter(iterable)
    R = [next(it) for i in range(r)]
    for i, item in enumerate(it, start=r+1):
      j = random.randrange(i)
      if j<r:
        R[j] = item
    return R

In [42]:
sample_size = 20000
sample_train_ix = reservoir_sampling(good_train_ix, sample_size)
logreg = sklearn.linear_model.LogisticRegression()

In [43]:
m = train_dataset.shape[1]*train_dataset.shape[2]
X = train_dataset[sample_train_ix].reshape(sample_size,m)
y = train_labels[sample_train_ix]
M = logreg.fit(X,y)

In [45]:
X_hat = test_dataset.reshape(test_dataset.shape[0],m)
y_hat = test_labels
L = M.score(X_hat, y_hat)
print(L)

0.7081
